In [4]:
import Levenshtein
from nltk import edit_distance
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd

In [6]:
# function to compute Levenshtein distance
def levenshtein_distance(str1, str2):
    distance = Levenshtein.distance(str1, str2)
    max_len = max(len(str1), len(str2))
    return distane / max_len

def n_gram_similarity(str1, str2):
    str1_ngrams = set(ngrams(str1, n))
    str2_ngrams = set(ngrams(str2, n))
    return len(str1_ngrams & str2_ngrams) / float(len(str1_ngrams | str2_ngrams))

def naro_winkler(str1, str2):
    return Levenshtein.jaro_winkler(str1, str2)

In [7]:
class HuggingFaceEmbedding:
    def __init__(self, model_name = "distilbert-base-uncased"):
        

SyntaxError: expected ':' (2132837906.py, line 2)